<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

### 1264. Page Recommendations

```mysql
SELECT DISTINCT page_id AS recommended_page
FROM likes
WHERE user_id IN (
    SELECT user2_id
    FROM friendship f1
    WHERE user1_id=1
    UNION
    SELECT user1_id
    FROM friendship f2
    WHERE user2_id=1
)
AND page_id NOT IN (
    SELECT page_id
    FROM likes
    WHERE user_id=1
)
```

### 1270. All People Report to the Given Manager

```mysql
-- BECAUSE 1 | boss | 1, So just need to use nested where managed_id IN (...)
SELECT DISTINCT employee_id
FROM employees
WHERE manager_id IN (
    SELECT employee_id
    FROM employees
    WHERE manager_id IN (
        SELECT employee_id
        FROM employees
        WHERE manager_id=1)
    ) AND employee_id!=1
```

### 1280. Students and Examinations

```mysql
-- group examinations first, more efficient
SELECT st.student_id, st.student_name, sub.subject_name,
    IFNULL(attended_exams, 0) AS attended_exams
FROM students st
CROSS JOIN subjects sub
LEFT JOIN (
    SELECT student_id, subject_name, COUNT(student_id) AS attended_exams
    FROM examinations
    GROUP BY student_id, subject_name) x
ON st.student_id=x.student_id AND sub.subject_name=x.subject_name
ORDER BY st.student_id, sub.subject_name

/*SELECT a.student_id, a.student_name, a.subject_name,
    IFNULL(COUNT(b.student_id), 0) AS attended_exams
FROM (
    SELECT student_id, student_name, subject_name
    FROM students st
    CROSS JOIN subjects sb) a
LEFT JOIN examinations b
ON a.student_id=b.student_id AND a.subject_name=b.subject_name
GROUP BY a.student_id, a.student_name, a.subject_name
ORDER BY a.student_id, a.subject_name*/
```

### 1285. Find the Start and End Number of Continuous Ranges

```mysql
-- MYSQL
/* can not select two columns from two table
has to cross join */
SELECT start_id, MIN(end_id) AS end_id
FROM (
    SELECT log_id AS start_id
    FROM logs
    WHERE log_id NOT IN (
        SELECT log_id+1
        FROM logs)
    ) a
CROSS JOIN
    (
    SELECT log_id AS end_id
    FROM logs
    WHERE log_id NOT IN (
        SELECT log_id-1
        FROM logs)
    ) b
ON end_id>=start_id
GROUP BY start_id
-- USE variables
/*SELECT start_id, end_id
FROM 
    (SELECT @i:=@i+1 id, log_id start_id
    FROM Logs, (SELECT @i:=0) n
    WHERE log_id-1 NOT IN (SELECT * FROM Logs)
    ) s
JOIN
    (SELECT @j:=@j+1 id, log_id end_id
    FROM Logs, (SELECT @j:=0) n
    WHERE log_id+1 NOT IN (SELECT * FROM Logs)
    ) e
ON s.id = e.id*/

-- SQL server
-- briliant use of group by
SELECT MIN(log_id) AS start_id, MAX(log_id) AS end_id
FROM (
    SELECT log_id, ROW_NUMBER() OVER(ORDER BY log_id) AS num
    FROM Logs) a
GROUP BY log_id - num
-- with cte, more concise code
-- use row_number, lead, lag
/*WITH cte AS
(SELECT 
log_id,
log_id-lag(log_id) OVER (ORDER BY log_id) AS lag_id,
log_id-lead(log_id) OVER (ORDER BY log_id) AS lead_id
FROM Logs)

SELECT 
a.log_id AS start_id,
b.log_id AS end_id
FROM
(SELECT log_id, row_number() OVER (ORDER BY log_id) AS rownum FROM cte WHERE lag_id>1 OR lag_id IS NULL) AS a
left join 
(SELECT log_id, row_number() OVER (ORDER BY log_id) AS rownum FROM cte WHERE lead_id<-1 OR lead_id IS NULL) AS b
ON a.rownum=b.rownum;*/

/*SELECT start_id, end_id
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY start_id) AS row, start_id
    FROM (
        SELECT start_id
        FROM (
            SELECT CASE WHEN ISNULL(lag_diff, 0)!=1 THEN log_id ELSE -1 END AS start_id
            FROM (
                SELECT log_id,
                    log_id-LAG(log_id) OVER (ORDER BY log_id) AS lag_diff
                FROM logs) a
            ) b
        WHERE start_id!=-1) c
    ) d
JOIN (
    SELECT ROW_NUMBER() OVER (ORDER BY end_id) AS row, end_id
    FROM (
        SELECT end_id
        FROM (
            SELECT CASE WHEN ISNULL(lead_diff, 0)!=-1 THEN log_id ELSE -1 END AS end_id
            FROM (
                SELECT log_id,
                    log_id-LEAD(log_id) OVER (ORDER BY log_id) AS lead_diff
                FROM logs) a1
            ) b1
        WHERE end_id!=-1) c1
    ) d1
ON d.row=d1.row*/
```

### 1294. Weather Type in Each Country

```mysql
SELECT c.country_name,
    CASE WHEN AVG(w.weather_state)>=25 THEN 'Hot'
        WHEN AVG(w.weather_state)<=15 THEN 'Cold'
        ELSE 'Warm' END AS weather_type
FROM countries c
JOIN (
    SELECT *
    FROM weather
    WHERE day BETWEEN '2019-11-01' AND '2019-11-30') w
ON c.country_id=w.country_id
GROUP BY c.country_name
```

### 1303. Find the Team Size

```mysql
SELECT employee_id, team_size
FROM employee e
JOIN (
    SELECT team_id, COUNT(employee_id) AS team_size
    FROM employee
    GROUP BY team_id) x
ON e.team_id=x.team_id
```

### 1308. Running Total for Different Genders

```mysql
-- MYSQL
SELECT 'F' AS gender, a.day, SUM(s1.score_points) AS total
FROM (
    SELECT *
    FROM scores
    WHERE gender='F') a
JOIN scores s1
ON a.gender=s1.gender AND a.day>=s1.day
GROUP BY a.day
UNION
SELECT 'M' AS gender, a.day, SUM(s1.score_points) AS total
FROM (
    SELECT *
    FROM scores
    WHERE gender='M') a
JOIN scores s1
ON a.gender=s1.gender AND a.day>=s1.day
GROUP BY a.day

-- SQL server
SELECT gender, day,
    SUM (score_points) OVER (PARTITION BY gender ORDER BY day) AS total
FROM scores
```

### 1321. Restaurant Growth

```mysql
-- MYSQL
SELECT a.visited_on, SUM(b.amount) AS amount,
    ROUND(SUM(b.amount)/7.0, 2) AS average_amount
FROM (
    SELECT visited_on, SUM(amount) AS amount
    FROM customer
    GROUP BY visited_on) a
JOIN  (
    SELECT visited_on, SUM(amount) AS amount
    FROM customer
    GROUP BY visited_on) b
ON b.visited_on<=a.visited_on AND a.visited_on<=ADDDATE(b.visited_on, INTERVAL 6 day)
    AND a.visited_on>=(
        SELECT ADDDATE(MIN(visited_on), INTERVAL 6 day)
        FROM customer
        )
GROUP BY a.visited_on

-- SQL server
/*SELECT visited_on, amount, ROUND(amount/7.0, 2) AS average_amount
FROM (
    SELECT visited_on,
        SUM(amount) OVER (ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS amount,
        ROW_NUMBER() OVER (ORDER BY visited_on) AS row
    FROM (
        SELECT visited_on, SUM(amount) AS amount
        FROM customer
        GROUP BY visited_on) a
    ) b
WHERE row>6*/
SELECT visited_on, amount, ROUND(amount/7.0, 2) AS average_amount
FROM (
    SELECT visited_on,
        SUM(amount) OVER (ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS amount
    FROM (
        SELECT visited_on, SUM(amount) AS amount
        FROM customer
        GROUP BY visited_on) a
    ) b
ORDER BY visited_on OFFSET 6 ROWS
```

### 1322. Ads Performance

```mysql
SELECT ad_id, IF(clicks+views=0, 0.00, ROUND(clicks/(clicks+views)*100, 2)) AS ctr
FROM (
    SELECT ad_id, SUM(clicks) AS clicks, SUM(views) AS views
    FROM (
        SELECT ad_id,
            CASE  WHEN action='clicked' THEN 1 ELSE 0 END AS clicks,
            CASE  WHEN action='viewed' THEN 1 ELSE 0 END AS views
        FROM ads) a
    GROUP BY ad_id) b
ORDER BY ctr DESC, ad_id
```

### 1327. List the Products Ordered in a Period

```mysql
SELECT p.product_name, o.unit
FROM (
    SELECT product_id, SUM(unit) AS unit
    FROM orders
    WHERE order_date BETWEEN '2020-02-01' AND '2020-02-29'
    GROUP BY product_id
    HAVING unit>=100) o
JOIN products p
ON o.product_id=p.product_id
```

### 1336. Number of Transactions per Visit

```mysql
-- MYSQL
-- missing data with visit but no transaction, need a way to generate a sequence
SELECT IFNULL(transaction_count, 0) as transactions_count, COUNT(*) AS visits_count
FROM visits v
LEFT JOIN (
    SELECT user_id, transaction_date, COUNT(*) AS transaction_count
    FROM transactions
    GROUP BY user_id, transaction_date) t
ON v.user_id=t.user_id AND v.visit_date=t.transaction_date
GROUP BY transaction_count

-- SQL server
with transaction_count AS (
    SELECT user_id, transaction_date, COALESCE(COUNT(*), 0) AS transaction_count
    FROM transactions
    GROUP BY user_id, transaction_date),
    seq as (
        SELECT 0 as row_num, max(transaction_count) as temp
        FROM transaction_count
        UNION ALL
        SELECT row_num+1 , temp FROM seq
        WHERE row_num < temp
    )

/*select *
from seq
WHERE transaction_count<=(SELECT COALESCE(MAX(transaction_count), 0) FROM transaction_count)*/
SELECT s.row_num AS transactions_count, IIF(x.visits_count IS NULL, 0, x.visits_count) AS visits_count
FROM seq s
LEFT JOIN (
    SELECT IIF(transaction_count IS NULL, 0, transaction_count) as transactions_count, COUNT(*) AS visits_count
    FROM visits v
    LEFT JOIN transaction_count t
    ON v.user_id=t.user_id AND v.visit_date=t.transaction_date
    GROUP BY transaction_count) x
ON s.row_num=x.transactions_count
WHERE row_num<=(SELECT COALESCE(MAX(transaction_count), 0) FROM transaction_count)
ORDER BY transactions_count
```

### 1341. Movie Rating

```mysql
-- MYSQL
-- simpler version
(
    SELECT u.name AS results
    FROM Movie_Rating mr
    LEFT JOIN Users u
    ON mr.user_id = u.user_id
    GROUP BY mr.user_id
    ORDER BY COUNT(mr.movie_id) DESC, u.name ASC
    LIMIT 1
)
UNION ALL
(
    SELECT m.title AS results
    FROM Movie_Rating mr
    LEFT JOIN Movies m
    ON mr.movie_id = m.movie_id
    WHERE substr(mr.created_at, 1, 7)='2020-02'
    GROUP BY mr.movie_id
    ORDER BY AVG(mr.rating) DESC, m.title ASC
    LIMIT 1
)
-- maybe faster
/*(SELECT name AS results
FROM users u
WHERE user_id IN (
    SELECT user_id
    FROM movie_rating
    GROUP BY user_id
    HAVING count(movie_id)=(
        SELECT MAX(id)
        FROM (
            SELECT COUNT(movie_id) AS id
            FROM movie_rating
            GROUP BY user_id
        ) x
    )
)
ORDER BY name
LIMIT 1)
UNION
(SELECT title AS results
FROM movies
WHERE movie_id in (
    SELECT movie_id
    FROM movie_rating
    WHERE substr(created_at, 1, 7)='2020-02'
    GROUP BY movie_id
    HAVING ROUND(AVG(rating), 6)=(
        SELECT MAX(avg_rating)
        FROM (
            SELECT movie_id, ROUND(AVG(rating), 6) AS avg_rating
            FROM movie_rating
            WHERE substr(created_at, 1, 7)='2020-02'
            GROUP BY movie_id
        ) x
    )
)
ORDER BY title
LIMIT 1)*/

-- SQL server
SELECT *
FROM (SELECT TOP 1
    name AS results
FROM users
WHERE user_id in (
    SELECT user_id
    FROM (
        SELECT user_id,
            RANK() OVER (ORDER BY rate_count DESC) AS rnk
        FROM (
            SELECT user_id, COUNT(rating) AS rate_count
            FROM movie_rating
            GROUP BY user_id) x
        ) y
    WHERE rnk=1)
ORDER BY name) a
UNION
SELECT *
FROM (SELECT TOP 1
    title AS results
FROM movies
WHERE movie_id in (
    SELECT movie_id
    FROM (
        SELECT movie_id,
            RANK() OVER (ORDER BY avg_rate DESC) AS rnk
        FROM (
            SELECT movie_id, AVG(rating*1.0) AS avg_rate
            FROM movie_rating
            WHERE LEFT(created_at, 7)='2020-02'
            GROUP BY movie_id) x
        ) y
    WHERE rnk=1
    )
ORDER BY title) b
```

### 1350. Students With Invalid Departments

```mysql
SELECT id, name
FROM students
WHERE department_id NOT IN (
    SELECT id
    FROM departments
)
```